In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
full = pd.read_csv('full.csv', names = ['sarc', 'not sarc'])
full

,sarc,not sarc
0,best day of my life,worst day of my life
1,best day of my life,worst day of my life
2,best day of my life,i had a horrible day
3,best day of my life,worst day of my life
4,best day of my life,i have never had a day worse than this in my ...
...,...,...
14965,if i were to describe myself in one word it w...,if i were to describe myself in one word it w...
14966,if i were to describe myself in one word it w...,if i were to describe myself in one word it w...
14967,if i were to describe myself in one word it w...,if i were to describe myself in one word it w...
14968,if i were to describe myself in one word it w...,if i were to describe myself in one word it w...


Task a: sarc, not sarc
sarcasm_ref: correct answer
human_aggregated: answer by human annotation
#human_sarcasm: number of people who agreed with human annotator

If at least 3 agree that the tweet is sarcastic, human_aggregated labeled 1.

In [3]:
eng_task_a = pd.read_csv('english_task_a.csv')
eng_task_a

,text,sarcasm_ref,human_aggregated,#humans_sarcasm
0,"Size on the the Toulouse team, That pack is mo...",0,0,1
1,Pinball!,0,0,0
2,So the Scottish Government want people to get ...,1,1,4
3,villainous pro tip : change the device name on...,0,0,0
4,I would date any of these men 🥺,0,0,1
...,...,...,...,...
1395,I’ve just seen this and felt it deserved a Ret...,0,0,0
1396,Omg how an earth is that a pen !!! 🤡,0,0,2
1397,Bringing Kanye and drake to a tl near you,0,0,2
1398,"I love it when women are referred to as ""girl ...",1,0,0


Given a sarcastic text and its non-sarcastic rephrase, i.e. two texts that convey the same meaning, determine which is the sarcastic one.

In [4]:
eng_task_c = pd.read_csv('english_task_c.csv')
eng_task_c
'I see that your team played well today!' in eng_task_c

False

In [5]:
task_A_test = pd.read_csv('task_A_En_test.csv')
task_A_test

,text,sarcastic
0,"Size on the the Toulouse team, That pack is mo...",0
1,Pinball!,0
2,So the Scottish Government want people to get ...,1
3,villainous pro tip : change the device name on...,0
4,I would date any of these men 🥺,0
...,...,...
1395,I’ve just seen this and felt it deserved a Ret...,0
1396,Omg how an earth is that a pen !!! 🤡,0
1397,Bringing Kanye and drake to a tl near you,0
1398,"I love it when women are referred to as ""girl ...",1


In [6]:
task_B_test = pd.read_csv('task_B_En_test.csv')
task_B_test

,text,sarcasm,irony,satire,understatement,overstatement,rhetorical_question
0,saw a video of someone getting a hug. would LO...,0,0,0,0,0,0
1,"""This Christmas I hope you all either get vacc...",0,0,0,0,0,0
2,It's the alamo,0,0,0,0,0,0
3,"Wind 5 mph E. Barometer 1029.8 hPa, Pressure t...",0,0,0,0,0,0
4,I constantly have loads of the new symptoms bu...,0,0,0,0,0,0
...,...,...,...,...,...,...,...
1395,Tempting to renew my membership and vote again...,0,0,0,0,0,0
1396,This week has felt like the longest in history...,0,0,0,0,0,0
1397,Of course it’s raining when I’m due to go out ...,0,0,0,0,0,0
1398,Weigh up a lie before you tell it.,0,0,0,0,0,0


In [7]:
task_C_test = pd.read_csv('task_C_En_test.csv')
task_C_test

,text_0,text_1,sarcastic_id
0,I see that your team played well today!,I'm sorry that your team didn't win yesterday.,0
1,"Anthony Taylor is such a fair referee, I wish ...",I hope Anthony Taylor is never put in charge o...,0
2,"the weather is gloomy, just raining and dull.",What a glorious weather today,1
3,People going out to get there boosters without...,Nice to see the sheep getting their boosters t...,1
4,"Really great weather we're having, love a bit ...",Really cold January so far - looking forward t...,0
...,...,...,...
195,"the tories betrayed the nation, what a surprise!","the tories betrayed the nation, as expected",0
196,Cant believe we have to spend the rest of our ...,Cant wait to spend the rest of my life waiting...,1
197,Isn't it just amazing how competent the govern...,"Everything is a total mess, how can anyone be ...",0
198,Thanks Boris Johnson for restricting travel ab...,The reasoning behind the tightening of travel ...,0


In [8]:
# 'I see that your team played well today!' is in task C
task_B_test['text'].eq('I see that your team played well today!').any()
task_C_test['text_0'].eq('I see that your team played well today!').any()


False

In [9]:
a_text = list(task_A_test['text'])
b_text = list(task_B_test['text'])
c_text = list(task_C_test['text_0'])
texts = list(set(a_text + b_text + c_text))
print(len(texts))
print(len(a_text + b_text + c_text))
texts

1503
3000


["Wasn't very found of education my whole life... became a teacher. Makes sense ",
 'Boris johnson must host the best parties',
 'Nice collective Boo for Boris during our set last night 👌🏼',
 "There will always be 'sensible' visitors coming to Spain.",
 'You look fantastic in that new dress.  It shows off your figure.',
 "Great Welsh Karaoke Rewrites (pt 94): 'Cyfarthfa Park is melting in the dark ...' 🎶🎙",
 '16:00 GMT: \r\nTemp: 9.5°C,\r\nWind: NNW, 8 mph (ave), 17 mph (gust),\r\nHumidity: 63%,\r\nRain (hourly) 1.5 mm,\r\nPressure: 1005 hPa, rising',
 "I think he's lost the plot 🤔",
 'Great to see you All it was a really fund Christmas celebration 🎉',
 'Drinks last night was a bit messy. May get some more beers tonight. Not sure though.',
 'Please I really want to watch Pepe..atleast make I see am live on the 21st.\r\nThis freeze out isn’t worth it at all',
 'I live for the great British weather!',
 'For my more youthful friends who are nearing the 6 months post-jab 2 time x',
 'Just 

In [10]:
train_En = pd.read_csv("train.En.csv")
train_En = train_En.rename(columns = {'Unnamed: 0': 'ID'})
train_En

,ID,tweet,sarcastic,rephrase,sarcasm,irony,satire,understatement,overstatement,rhetorical_question
0,0,The only thing I got from college is a caffein...,1,"College is really difficult, expensive, tiring...",0.0,1.0,0.0,0.0,0.0,0.0
1,1,I love it when professors draw a big question ...,1,I do not like when professors don’t write out ...,1.0,0.0,0.0,0.0,0.0,0.0
2,2,Remember the hundred emails from companies whe...,1,"I, at the bare minimum, wish companies actuall...",0.0,1.0,0.0,0.0,0.0,0.0
3,3,Today my pop-pop told me I was not “forced” to...,1,"Today my pop-pop told me I was not ""forced"" to...",1.0,0.0,0.0,0.0,0.0,0.0
4,4,@VolphanCarol @littlewhitty @mysticalmanatee I...,1,I would say Ted Cruz is an asshole and doesn’t...,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
3463,3463,The population spike in Chicago in 9 months is...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3464,3464,You'd think in the second to last English clas...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3465,3465,I’m finally surfacing after a holiday to Scotl...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3466,3466,Couldn't be prouder today. Well done to every ...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
filtered_train_EN = train_En[["ID", "tweet", "sarcastic", "rephrase"]]
filtered_train_EN["ID"] = filtered_train_EN["ID"].map(lambda x : "train_" + str(x))
filtered_train_EN

/tmp/ipykernel_248513/3519942343.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_train_EN["ID"] = filtered_train_EN["ID"].map(lambda x : "train_" + str(x))


,ID,tweet,sarcastic,rephrase
0,train_0,The only thing I got from college is a caffein...,1,"College is really difficult, expensive, tiring..."
1,train_1,I love it when professors draw a big question ...,1,I do not like when professors don’t write out ...
2,train_2,Remember the hundred emails from companies whe...,1,"I, at the bare minimum, wish companies actuall..."
3,train_3,Today my pop-pop told me I was not “forced” to...,1,"Today my pop-pop told me I was not ""forced"" to..."
4,train_4,@VolphanCarol @littlewhitty @mysticalmanatee I...,1,I would say Ted Cruz is an asshole and doesn’t...
...,...,...,...,...
3463,train_3463,The population spike in Chicago in 9 months is...,0,NaN
3464,train_3464,You'd think in the second to last English clas...,0,NaN
3465,train_3465,I’m finally surfacing after a holiday to Scotl...,0,NaN
3466,train_3466,Couldn't be prouder today. Well done to every ...,0,NaN


In [12]:
task_A_test
filtered_train_EN_dict = {k: list(filtered_train_EN[k]) for k in filtered_train_EN}
filtered_train_EN_dict

{'ID': ['train_0',
  'train_1',
  'train_2',
  'train_3',
  'train_4',
  'train_5',
  'train_6',
  'train_7',
  'train_8',
  'train_9',
  'train_10',
  'train_11',
  'train_12',
  'train_13',
  'train_14',
  'train_15',
  'train_16',
  'train_17',
  'train_18',
  'train_19',
  'train_20',
  'train_21',
  'train_22',
  'train_23',
  'train_24',
  'train_25',
  'train_26',
  'train_27',
  'train_28',
  'train_29',
  'train_30',
  'train_31',
  'train_32',
  'train_33',
  'train_34',
  'train_35',
  'train_36',
  'train_37',
  'train_38',
  'train_39',
  'train_40',
  'train_41',
  'train_42',
  'train_43',
  'train_44',
  'train_45',
  'train_46',
  'train_47',
  'train_48',
  'train_49',
  'train_50',
  'train_51',
  'train_52',
  'train_53',
  'train_54',
  'train_55',
  'train_56',
  'train_57',
  'train_58',
  'train_59',
  'train_60',
  'train_61',
  'train_62',
  'train_63',
  'train_64',
  'train_65',
  'train_66',
  'train_67',
  'train_68',
  'train_69',
  'train_70',
  'train_7

In [13]:
for idx, row in task_A_test.iterrows():
    # print(filtered_train_EN_dict['sarcastic'])
    if row['text'] not in filtered_train_EN_dict['tweet'] :
        filtered_train_EN_dict['tweet'].append(row['text'])
        filtered_train_EN_dict['sarcastic'].append(row['sarcastic'])
        filtered_train_EN_dict['rephrase'].append(None)
        filtered_train_EN_dict['ID'].append('test_a_' + str(idx))

In [14]:
len(filtered_train_EN_dict['ID'])

4862

In [15]:
task_B_test

,text,sarcasm,irony,satire,understatement,overstatement,rhetorical_question
0,saw a video of someone getting a hug. would LO...,0,0,0,0,0,0
1,"""This Christmas I hope you all either get vacc...",0,0,0,0,0,0
2,It's the alamo,0,0,0,0,0,0
3,"Wind 5 mph E. Barometer 1029.8 hPa, Pressure t...",0,0,0,0,0,0
4,I constantly have loads of the new symptoms bu...,0,0,0,0,0,0
...,...,...,...,...,...,...,...
1395,Tempting to renew my membership and vote again...,0,0,0,0,0,0
1396,This week has felt like the longest in history...,0,0,0,0,0,0
1397,Of course it’s raining when I’m due to go out ...,0,0,0,0,0,0
1398,Weigh up a lie before you tell it.,0,0,0,0,0,0


In [16]:
for idx, row in task_B_test.iterrows():
    # print(filtered_train_EN_dict['sarcastic'])
    if row['text'] not in filtered_train_EN_dict['tweet'] :
        filtered_train_EN_dict['tweet'].append(row['text'])
        filtered_train_EN_dict['sarcastic'].append(1)
        filtered_train_EN_dict['rephrase'].append(None)
        filtered_train_EN_dict['ID'].append('test_b_' + str(idx))

In [17]:
len(filtered_train_EN_dict['ID'])

4862

In [18]:
task_C_test

,text_0,text_1,sarcastic_id
0,I see that your team played well today!,I'm sorry that your team didn't win yesterday.,0
1,"Anthony Taylor is such a fair referee, I wish ...",I hope Anthony Taylor is never put in charge o...,0
2,"the weather is gloomy, just raining and dull.",What a glorious weather today,1
3,People going out to get there boosters without...,Nice to see the sheep getting their boosters t...,1
4,"Really great weather we're having, love a bit ...",Really cold January so far - looking forward t...,0
...,...,...,...
195,"the tories betrayed the nation, what a surprise!","the tories betrayed the nation, as expected",0
196,Cant believe we have to spend the rest of our ...,Cant wait to spend the rest of my life waiting...,1
197,Isn't it just amazing how competent the govern...,"Everything is a total mess, how can anyone be ...",0
198,Thanks Boris Johnson for restricting travel ab...,The reasoning behind the tightening of travel ...,0


In [19]:
for idx, row in task_C_test.iterrows():
    # print(filtered_train_EN_dict['sarcastic'])
    sarc_id = row['sarcastic_id']
    if row['text_0'] not in filtered_train_EN_dict['tweet'] :
        filtered_train_EN_dict['tweet'].append(row['text_0'])
        filtered_train_EN_dict['sarcastic'].append(1 - sarc_id)
        filtered_train_EN_dict['rephrase'].append(None if sarc_id == 1 else row['text_1'])
        filtered_train_EN_dict['ID'].append('test_c_' + str(idx))

In [20]:
for idx, row in full.iterrows():
    filtered_train_EN_dict['tweet'].append(row['sarc'])
    filtered_train_EN_dict['sarcastic'].append(1)
    filtered_train_EN_dict['rephrase'].append(row['not sarc'])
    filtered_train_EN_dict['ID'].append('sign_' + str(idx))

In [21]:
final = pd.DataFrame(filtered_train_EN_dict)
final['sarcastic'].sum()

16099

In [22]:
final

,ID,tweet,sarcastic,rephrase
0,train_0,The only thing I got from college is a caffein...,1,"College is really difficult, expensive, tiring..."
1,train_1,I love it when professors draw a big question ...,1,I do not like when professors don’t write out ...
2,train_2,Remember the hundred emails from companies whe...,1,"I, at the bare minimum, wish companies actuall..."
3,train_3,Today my pop-pop told me I was not “forced” to...,1,"Today my pop-pop told me I was not ""forced"" to..."
4,train_4,@VolphanCarol @littlewhitty @mysticalmanatee I...,1,I would say Ted Cruz is an asshole and doesn’t...
...,...,...,...,...
19936,sign_14965,if i were to describe myself in one word it w...,1,if i were to describe myself in one word it w...
19937,sign_14966,if i were to describe myself in one word it w...,1,if i were to describe myself in one word it w...
19938,sign_14967,if i were to describe myself in one word it w...,1,if i were to describe myself in one word it w...
19939,sign_14968,if i were to describe myself in one word it w...,1,if i were to describe myself in one word it w...


In [23]:
final.to_csv('unshuffled.csv')

In [31]:
import random

unshuffled = pd.read_csv('unshuffled.csv').rename(columns = {'Unnamed: 0' : 'ID', 'ID' : 'original_ID'})
shuffled = unshuffled.sample(frac= 1).copy()

shuffling = list(x for x in range(len(shuffled)))
random.shuffle(shuffling)

shuffled['ID'] = shuffled['ID'].map(lambda x : str(int(shuffling[x])))

shuffled = shuffled.sample(frac= 1)
if len(shuffled) != len(unshuffled) :
    print(f"WARNING! : {len(shuffled)} != {len(unshuffled)}")
shuffled.to_csv('shuffled.csv', index=False)

In [33]:
from tqdm import tqdm 

base_length = len(shuffled)
for i in tqdm(range(base_length)) :
    tweet = shuffled["tweet"][i]
    if tweet in set( shuffled["tweet"][:i+1] ) :
        shuffled = shuffled.drop(i, axis=0)


total_length = len(shuffled)
print(f"The length of the data is : {total_length}")

groups = []

sources = shuffled['original_ID'].map(lambda x : x.split('_')[0])

for val in set(shuffled['sarcastic']) :
    groups.append( shuffled.loc[ (shuffled['sarcastic'] == val) & ( sources == 'sign' ) ] )
    groups.append( shuffled.loc[ (shuffled['sarcastic'] == val) & ( sources != 'sign' ) ] )

print(f"There are {len(groups)} groups")
print(f"Their length is : {[len(x) for x in groups]}")
groups = [x for x in groups if len(x) > 0] # remove empty group
print(f"There are {len(groups)} groups")
print(f"Their length is : {[len(x) for x in groups]}")

subsamples = []
n = 1000
for grp in groups :
    sample_size = int(n * len(grp) / total_length)
    subsamples.append( grp.sample(n=sample_size, random_state=42) )

print(f"Their sub length is : {[len(x) for x in subsamples]}")
sample = pd.concat(subsamples)
sample = sample.sample(frac=1) # shuffles the sample
sample.to_csv("sample_with_answers_v2.csv", index=False)
sample[["ID", "tweet"]].to_csv("sample_to_annotate_v2.csv", index=False)

100%|████████████████████████████████| 19941/19941 [00:49<00:00, 406.43it/s]

The length of the data is : 5561
There are 4 groups
Their length is : [0, 3215, 1305, 1041]
There are 3 groups
Their length is : [3215, 1305, 1041]
Their sub length is : [578, 234, 187]
